<a href="https://colab.research.google.com/github/shreyasvinaya/SAiDL-Summer-Assignment-2023/blob/main/SAIDL_assignment_task2_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://hasocfire.github.io/hasoc/2019/files/hindi_dataset.zip
!unzip hindi_dataset.zip

--2023-05-31 20:49:43--  https://hasocfire.github.io/hasoc/2019/files/hindi_dataset.zip
Resolving hasocfire.github.io (hasocfire.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to hasocfire.github.io (hasocfire.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 667717 (652K) [application/zip]
Saving to: ‘hindi_dataset.zip.2’

hindi_dataset.zip.2 100%[===================>] 652.07K  --.-KB/s    in 0.006s  

2023-05-31 20:49:43 (115 MB/s) - ‘hindi_dataset.zip.2’ saved [667717/667717]

Archive:  hindi_dataset.zip
replace __MACOSX/._hindi_dataset? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
!pip install transformers==4.28.0
!pip install googletrans


In [3]:
import pandas as pd
import random
from googletrans import Translator
from transformers import BertTokenizer, BertForSequenceClassification, MBartForConditionalGeneration, MBart50TokenizerFast, Trainer, TrainingArguments
import torch
from sklearn.metrics import accuracy_score


In [4]:
import pandas as pd
import re

# Load HASOC 2019 Hindi dataset
data = pd.read_csv("/content/hindi_dataset/hindi_dataset.tsv", sep = '\t')

# Data cleaning function
def clean_text(text):
    # Remove special characters, URLs, and irrelevant information
    text = re.sub(r"http\S+|www\S+|https\S+", "", text, flags=re.MULTILINE)

    text = re.sub(r'@\S+', '', text)
    text = re.sub(r'#\S+', '', text)
    text = text.strip()
    
    return text

# Clean the dataset
data["cleaned_text"] = data["text"].apply(clean_text)



In [5]:
data_tasks = pd.get_dummies(data[['task_1', 'task_2', 'task_3']])
data = data[['text_id', 'text']]
# concatenate the text and task DataFrames
data = pd.concat([data, data_tasks], axis=1)


In [6]:
data.describe()

,task_1_HOF,task_1_NOT,task_2_HATE,task_2_NONE,task_2_OFFN,task_2_PRFN,task_3_NONE,task_3_TIN,task_3_UNT
count,4665.000000,4665.000000,4665.000000,4665.000000,4665.000000,4665.000000,4665.000000,4665.000000,4665.000000
mean,0.529260,0.470740,0.119185,0.470740,0.144909,0.265166,0.470740,0.331190,0.198071
std,0.499197,0.499197,0.324041,0.499197,0.352047,0.441469,0.499197,0.470692,0.398588
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [7]:
data_sample = data.head(20)

In [8]:
data_sample.head()

,text_id,text,task_1_HOF,task_1_NOT,task_2_HATE,task_2_NONE,task_2_OFFN,task_2_PRFN,task_3_NONE,task_3_TIN,task_3_UNT
0,hasoc_hi_5556,"बांग्लादेश की शानदार वापसी, भारत को 314 रन पर ...",0,1,0,1,0,0,1,0,0
1,hasoc_hi_5648,सब रंडी नाच देखने मे व्यस्त जैसे ही कोई #शांती...,1,0,0,0,0,1,0,0,1
2,hasoc_hi_164,तुम जैसे हरामियों के लिए बस जूतों की कमी है शु...,1,0,0,0,0,1,0,1,0
3,hasoc_hi_3530,"बीजेपी MLA आकाश विजयवर्गीय जेल से रिहा, जमानत ...",0,1,0,1,0,0,1,0,0
4,hasoc_hi_5206,चमकी बुखार: विधानसभा परिसर में आरजेडी का प्रदर...,0,1,0,1,0,0,1,0,0


In [ ]:


from googletrans import Translator
import random

not_trans=[]

def code_mix(text, target_language, cmi):
    try:
        translator = Translator(service_urls=['translate.google.com'])
        translated_text = translator.translate(text, src='hi', dest=target_language).text
    except Exception as e:
        print(f"Error during translation: {e}")
        print(data_sample.index[data_sample['text'] == text].tolist())
        not_trans.append(data_sample.index[data_sample['text'] == text].tolist()[0])
        return text
    words = text.split()
    translated_words = translated_text.split()

    mixed_words = []
    for word, translated_word in zip(words, translated_words):
        if random.random() < cmi:
            mixed_words.append(translated_word)
        else:
            mixed_words.append(word)
    
    return " ".join(mixed_words)


target_language = 'en'
cmi_levels = [0.2, 0.4, 0.6]

# Create code-mixed data_sample for each CMI level
for cmi in cmi_levels:
    data_sample[f"cmi_{cmi}"] = data_sample["text"].apply(lambda x: code_mix(x, target_language, cmi))


In [10]:
data_sample = data_sample.drop(columns=['task_2_NONE','task_3_NONE'])


In [11]:
cols = data_sample.columns[0:7]
label_cols = ['task_1_HOF', 'task_1_NOT', 'task_2_HATE', 'task_2_OFFN', 'task_2_PRFN']

data_sample['labels'] = list(data_sample[label_cols].values)
data_sample.head()

,text_id,text,task_1_HOF,task_1_NOT,task_2_HATE,task_2_OFFN,task_2_PRFN,task_3_TIN,task_3_UNT,cmi_0.2,cmi_0.4,cmi_0.6,labels
0,hasoc_hi_5556,"बांग्लादेश की शानदार वापसी, भारत को 314 रन पर ...",0,1,0,0,0,0,0,"बांग्लादेश की शानदार वापसी, भारत को 314 runs प...","बांग्लादेश brilliant comeback, वापसी, भारत को ...","Bangladesh's की शानदार वापसी, भारत को 314 runs...","[0, 1, 0, 0, 0]"
1,hasoc_hi_5648,सब रंडी नाच देखने मे व्यस्त जैसे ही कोई #शांती...,1,0,0,0,1,0,1,सब रंडी नाच देखने मे व्यस्त जैसे ही कोई #शांती...,सब रंडी नाच देखने is व्यस्त watching all कोई d...,सब रंडी नाच देखने मे व्यस्त जैसे ही कोई #शांती...,"[1, 0, 0, 0, 1]"
2,hasoc_hi_164,तुम जैसे हरामियों के लिए बस जूतों की कमी है शु...,1,0,0,0,1,1,0,तुम जैसे हरामियों के लिए बस जूतों की कमी है शु...,There is हरामियों के लिए बस जूतों की कमी like ...,तुम जैसे हरामियों के लिए बस जूतों की कमी है शु...,"[1, 0, 0, 0, 1]"
3,hasoc_hi_3530,"बीजेपी MLA आकाश विजयवर्गीय जेल से रिहा, जमानत ...",0,1,0,0,0,0,0,बीजेपी MLA आकाश Vijayvargiya released from रिह...,"बीजेपी MLA आकाश विजयवर्गीय जेल से रिहा, जमानत ...","BJP MLA Akash Vijayvargiya released from jail,...","[0, 1, 0, 0, 0]"
4,hasoc_hi_5206,चमकी बुखार: विधानसभा परिसर में आरजेडी का प्रदर...,0,1,0,0,0,0,0,चमकी बुखार: विधानसभा परिसर में आरजेडी का प्रदर...,चमकी बुखार: विधानसभा परिसर में आरजेडी का premi...,"Shining fever: RJD परिसर in the का premises, त...","[0, 1, 0, 0, 0]"


In [12]:
labels = data_sample.labels.values
labels

array([array([0, 1, 0, 0, 0], dtype=uint8),
       array([1, 0, 0, 0, 1], dtype=uint8),
       array([1, 0, 0, 0, 1], dtype=uint8),
       array([0, 1, 0, 0, 0], dtype=uint8),
       array([0, 1, 0, 0, 0], dtype=uint8),
       array([0, 1, 0, 0, 0], dtype=uint8),
       array([0, 1, 0, 0, 0], dtype=uint8),
       array([0, 1, 0, 0, 0], dtype=uint8),
       array([0, 1, 0, 0, 0], dtype=uint8),
       array([0, 1, 0, 0, 0], dtype=uint8),
       array([0, 1, 0, 0, 0], dtype=uint8),
       array([0, 1, 0, 0, 0], dtype=uint8),
       array([0, 1, 0, 0, 0], dtype=uint8),
       array([0, 1, 0, 0, 0], dtype=uint8),
       array([0, 1, 0, 0, 0], dtype=uint8),
       array([0, 1, 0, 0, 0], dtype=uint8),
       array([0, 1, 0, 0, 0], dtype=uint8),
       array([0, 1, 0, 0, 0], dtype=uint8),
       array([1, 0, 0, 1, 0], dtype=uint8),
       array([0, 1, 0, 0, 0], dtype=uint8)], dtype=object)

In [13]:
data_neat = data_sample[['text_id', 'text','cmi_0.2', 'cmi_0.4', 'cmi_0.6', 'labels']]
data_neat.head()

,text_id,text,cmi_0.2,cmi_0.4,cmi_0.6,labels
0,hasoc_hi_5556,"बांग्लादेश की शानदार वापसी, भारत को 314 रन पर ...","बांग्लादेश की शानदार वापसी, भारत को 314 runs प...","बांग्लादेश brilliant comeback, वापसी, भारत को ...","Bangladesh's की शानदार वापसी, भारत को 314 runs...","[0, 1, 0, 0, 0]"
1,hasoc_hi_5648,सब रंडी नाच देखने मे व्यस्त जैसे ही कोई #शांती...,सब रंडी नाच देखने मे व्यस्त जैसे ही कोई #शांती...,सब रंडी नाच देखने is व्यस्त watching all कोई d...,सब रंडी नाच देखने मे व्यस्त जैसे ही कोई #शांती...,"[1, 0, 0, 0, 1]"
2,hasoc_hi_164,तुम जैसे हरामियों के लिए बस जूतों की कमी है शु...,तुम जैसे हरामियों के लिए बस जूतों की कमी है शु...,There is हरामियों के लिए बस जूतों की कमी like ...,तुम जैसे हरामियों के लिए बस जूतों की कमी है शु...,"[1, 0, 0, 0, 1]"
3,hasoc_hi_3530,"बीजेपी MLA आकाश विजयवर्गीय जेल से रिहा, जमानत ...",बीजेपी MLA आकाश Vijayvargiya released from रिह...,"बीजेपी MLA आकाश विजयवर्गीय जेल से रिहा, जमानत ...","BJP MLA Akash Vijayvargiya released from jail,...","[0, 1, 0, 0, 0]"
4,hasoc_hi_5206,चमकी बुखार: विधानसभा परिसर में आरजेडी का प्रदर...,चमकी बुखार: विधानसभा परिसर में आरजेडी का प्रदर...,चमकी बुखार: विधानसभा परिसर में आरजेडी का premi...,"Shining fever: RJD परिसर in the का premises, त...","[0, 1, 0, 0, 0]"


In [ ]:
%pip uninstall -y accelerate
%pip install git+https://github.com/huggingface/accelerate

PS: DONT RUN THE BELOW CELL MULTIPLE TIMES WITHOUT WIPING THE COLAB VM, IT WILL RUN OUT OF DISK SPACE

In [29]:
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Load pre-trained BERT and mBERT models and tokenizers
tokenizer_bert = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
model_bert = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=5)

tokenizer_mbert = AutoTokenizer.from_pretrained("google/muril-base-cased")
model_mbert = AutoModelForSequenceClassification.from_pretrained("google/muril-base-cased", num_labels=5)

def encode_text(text, tokenizer, max_length=256):
    return tokenizer(text, truncation=True, max_length=max_length, padding='max_length', return_tensors='pt')

# Create a custom dataset class for the data
class CodeMixedDataset(torch.utils.data.Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        label = np.argmax(label)
        encoding = encode_text(text, self.tokenizer)
        return {**{key: torch.squeeze(val, 0) for key, val in encoding.items()}, 'labels': torch.tensor(label).long()}


# Split the dataset into training and validation sets
train_data, val_data = train_test_split(data_sample, test_size=0.2)

# Create dataset objects for training and validation
train_dataset = CodeMixedDataset(train_data["cmi_0.4"].tolist(), train_data["labels"].tolist(), tokenizer_bert)
val_dataset = CodeMixedDataset(val_data["cmi_0.4"].tolist(), val_data["labels"].tolist(), tokenizer_bert)



def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {'accuracy': accuracy_score(labels, predictions)}


# Define Trainer and TrainingArguments for BERT model (will GPU run out of memory?)
training_args_bert = TrainingArguments(
    output_dir='./results_bert',
    num_train_epochs=10,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs_bert',
    logging_steps=1,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    evaluation_strategy="epoch",
    save_strategy="epoch" 
)


trainer_bert = Trainer(
    model=model_bert,
    args=training_args_bert,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

# Finetune BERT model on the training data
trainer_bert.train()

# Save the finetuned BERT model
trainer_bert.save_model("./fine_tuned_bert")

# Clearing Cache so that colab doesnt freak out
torch.cuda.empty_cache()

# Repeat the process for mBERT model and hope GPU doesnt run out of memory
train_mdataset = CodeMixedDataset(train_data["cmi_0.4"].tolist(), train_data["labels"].tolist(), tokenizer_mbert)
val_mdataset = CodeMixedDataset(val_data["cmi_0.4"].tolist(), val_data["labels"].tolist(), tokenizer_mbert)

training_args_mbert = TrainingArguments(
    output_dir='./results_mbert',
    num_train_epochs=10,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs_mbert',
    logging_steps=1,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    evaluation_strategy="epoch",
    save_strategy="epoch" 
)
trainer_mbert = Trainer(
    model=model_mbert,
    args=training_args_mbert,
    train_dataset=train_mdataset,
    eval_dataset=val_mdataset,
    compute_metrics=compute_metrics
)

# Finetune mBERT model on the training data
trainer_mbert.train()

# Save the finetuned mBERT model
trainer_mbert.save_model("./fine_tuned_mbert")


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

Epoch,Training Loss,Validation Loss,Accuracy
1,1.718100,1.587680,0.250000
2,1.673000,1.586024,0.250000
3,1.676100,1.581883,0.250000
4,1.613700,1.575003,0.250000
5,1.653100,1.568058,0.500000
6,1.637500,1.562680,0.500000
7,1.622000,1.557980,0.500000
8,1.610100,1.553049,0.500000
9,1.669000,1.548299,0.500000
10,1.604800,1.542501,0.500000


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,1.610400,1.609441,0.250000
2,1.610800,1.609435,0.250000
3,1.610900,1.609423,0.250000
4,1.610900,1.609406,0.250000
5,1.610200,1.609383,0.250000
6,1.610800,1.609354,0.250000
7,1.610300,1.609320,0.250000
8,1.610600,1.609280,0.250000
9,1.610200,1.609234,0.250000
10,1.610600,1.609182,0.250000


In [28]:
torch.cuda.empty_cache()

In [30]:
# Evaluate the model on the test data
test_results = trainer_mbert.evaluate(val_dataset)

# Print out the results
print(test_results)


{'eval_loss': 1.609440803527832, 'eval_accuracy': 0.25, 'eval_runtime': 0.1061, 'eval_samples_per_second': 37.686, 'eval_steps_per_second': 9.421, 'epoch': 10.0}
